---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [4]:
# count = 1
# graph_dicts = {}
# for graph in P1_Graphs:
#     degree_unique_vals = sorted(set(graph.degree().values()), reverse=True)
#     graph_dicts['graph-'+str(count)+'-degree-distribution'] = [(i, list(graph.degree().values()).count(i)/float(graph.number_of_nodes())) for i in degree_unique_vals]
#     graph_dicts['graph-'+str(count)+'-clustering'] = nx.average_clustering(graph)
#     graph_dicts['graph-'+str(count)+'-node_count'] = graph.number_of_nodes()
#     graph_dicts['graph-'+str(count)+'-shortest-path'] = nx.average_shortest_path_length(graph)
#     count=count+1

In [21]:
# degree distribution analysis
# graph_dicts['graph-4-degree-distribution']

[(92, 0.0013333333333333333),
 (62, 0.0013333333333333333),
 (55, 0.0013333333333333333),
 (53, 0.0013333333333333333),
 (51, 0.0013333333333333333),
 (47, 0.0026666666666666666),
 (46, 0.004),
 (44, 0.0026666666666666666),
 (42, 0.0026666666666666666),
 (36, 0.0013333333333333333),
 (32, 0.0026666666666666666),
 (31, 0.0026666666666666666),
 (30, 0.0013333333333333333),
 (29, 0.004),
 (27, 0.008),
 (25, 0.0013333333333333333),
 (24, 0.0026666666666666666),
 (23, 0.0013333333333333333),
 (22, 0.008),
 (21, 0.005333333333333333),
 (20, 0.0013333333333333333),
 (19, 0.0013333333333333333),
 (18, 0.005333333333333333),
 (17, 0.012),
 (16, 0.006666666666666667),
 (15, 0.009333333333333334),
 (14, 0.010666666666666666),
 (13, 0.018666666666666668),
 (12, 0.02),
 (11, 0.017333333333333333),
 (10, 0.025333333333333333),
 (9, 0.03333333333333333),
 (8, 0.06266666666666666),
 (7, 0.07733333333333334),
 (6, 0.11866666666666667),
 (5, 0.17733333333333334),
 (4, 0.3453333333333333)]

In [11]:
# # clustering distribution analysis
# for i in range(1,6):
#     print(graph_dicts['graph-{}-clustering'.format(i)])

0.03167539146454044
0.5642419635919628
0.4018222222222227
0.03780379975223251
0.0033037037037037037


In [12]:
# # shortest path distribution analysis
# for i in range(1,6):
#     print(graph_dicts['graph-{}-shortest-path'.format(i)])

4.099161161161161
5.089871871871872
9.378702269692925
3.1048046283934134
5.0785509568313305


In [14]:
# # node count in each graph
# for i in range(1,6):
#     print(graph_dicts['graph-{}-node_count'.format(i)])

1000
1000
750
750
750


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [ ]:
def graph_identification():
    
    # Your Code Here
    
    return ['SW_L','SW_H','SW_H','SW_L','PA']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [104]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [139]:
nodes = G.nodes(data=True)
df = pd.DataFrame(index = G.nodes())
df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['management-salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['degrees'] = pd.Series(nx.degree(G))
df['clustering'] = pd.Series(nx.clustering(G))
# node_attributes_from_graph = [(node[0], node[1]['Department'], node[1]['ManagementSalary']) for node in nodes]

In [140]:
def mgmt_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    mgmt_emails = 0
    for edge in edges:
        if salary_type_dict[edge[1]] == 1:
            mgmt_emails=mgmt_emails+1
    row['mgmt_communication'] = mgmt_emails
    return row



In [141]:
def non_mgmt_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    non_mgmt_emails = 0
    for edge in edges:
        if salary_type_dict[edge[1]] == 0:
            non_mgmt_emails=non_mgmt_emails+1
    row['non_mgmt_communication'] = non_mgmt_emails
    return row

In [142]:
def unknown_emails(row, salary_type_dict):
    edges = G.edges(row['nodes'])
    unknown_emails = 0
    for edge in edges:
        if np.isnan(salary_type_dict[edge[1]]):
            unknown_emails=unknown_emails+1
    row['unknown_communication'] = unknown_emails
    return row

In [143]:
df = (df.reset_index()
        .rename(columns={'index': 'nodes'})
        .apply(mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
        .apply(non_mgmt_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1)
        .apply(unknown_emails,args=(nx.get_node_attributes(G,'ManagementSalary'),), axis=1))

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 8 columns):
nodes                     1005 non-null float64
department                1005 non-null float64
management-salary         753 non-null float64
degrees                   1005 non-null float64
clustering                1005 non-null float64
mgmt_communication        1005 non-null float64
non_mgmt_communication    1005 non-null float64
unknown_communication     1005 non-null int64
dtypes: float64(7), int64(1)
memory usage: 62.9 KB


In [148]:
df_training = df[df['management-salary'].isnull() == False]
df_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 0 to 1004
Data columns (total 8 columns):
nodes                     753 non-null float64
department                753 non-null float64
management-salary         753 non-null float64
degrees                   753 non-null float64
clustering                753 non-null float64
mgmt_communication        753 non-null float64
non_mgmt_communication    753 non-null float64
unknown_communication     753 non-null int64
dtypes: float64(7), int64(1)
memory usage: 52.9 KB


In [149]:
df_prediction = df[df['management-salary'].isnull()]
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1 to 1001
Data columns (total 8 columns):
nodes                     252 non-null float64
department                252 non-null float64
management-salary         0 non-null float64
degrees                   252 non-null float64
clustering                252 non-null float64
mgmt_communication        252 non-null float64
non_mgmt_communication    252 non-null float64
unknown_communication     252 non-null int64
dtypes: float64(7), int64(1)
memory usage: 17.7 KB


In [ ]:
from sklearn.model_selection import train_test_split
train_test_split?
X_train, X_test, y_train, y_test = train_test_split(df_training.loc[:, df_])

In [ ]:
def salary_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here